#Install pytorch and load libraries/imports


In [ ]:
Batches

In [ ]:
pip install torch

In [ ]:
pip install Cython

#Connect to tiingo and load in MSFT daily data

In [ ]:
#library imports
import pandas as pd
import numpy as np

In [ ]:
# Mount google drive 
# (go to the URL and copy/paste the code there into the box that appears below this code cell)

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#install tingo
!pip install tiingo # https://tiingo-python.readthedocs.io/en/latest/readme.html

In [ ]:
import tiingo
from tiingo import TiingoClient

config = {}

# To reuse the same HTTP Session across API calls (and have better performance), include a session key.
config['session'] = True

# If you don't have your API key as an environment variable,
# pass it in via a configuration dictionary.
config['api_key'] = ""

# Initialize
client = TiingoClient(config)

In [ ]:
msft_ticker_history = client.get_dataframe('MSFT',
                                      frequency='daily',
                                      startDate='1986-03-13',
                                      endDate='2020-05-29')

In [ ]:
msft_ticker_history
msft_ticker_history = msft_ticker_history[7000:]

In [ ]:
#Need to label the rows for classification, i.e. buy/sell/hold, up/down/nochange
#Could add previous days as columns but should first run just like this
#If open at t2 is greater than t1 - buy/up
#If open at t2 is less than t1 - sell/down
#If open at t2 = t1 - hold/nochange

msft_ticker_history['open_nextday'] = msft_ticker_history['open'].shift(-1)
msft_ticker_history

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,open_nextday
date,,,,,,,,,,,,,
2013-12-16 00:00:00+00:00,36.885,37.00,36.5400,36.7300,31734200,31.826906,31.926135,31.529216,31.693161,31734200,0.0,1.0,36.9400
2013-12-17 00:00:00+00:00,36.520,37.11,36.3344,36.9400,45687700,31.511959,32.021051,31.351810,31.874363,45687700,0.0,1.0,36.3601
2013-12-18 00:00:00+00:00,36.580,36.60,35.5300,36.3601,63192100,31.563731,31.580988,30.657719,31.373986,63192100,0.0,1.0,36.5100
2013-12-19 00:00:00+00:00,36.250,36.55,36.0800,36.5100,34160100,31.278984,31.537845,31.132296,31.503330,34160100,0.0,1.0,36.2000
2013-12-20 00:00:00+00:00,36.800,36.93,36.1850,36.2000,62649100,31.753562,31.865735,31.222898,31.235841,62649100,0.0,1.0,36.8100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-22 00:00:00+00:00,183.510,184.46,182.5400,183.1900,20826898,182.162652,183.105677,181.199774,181.845002,20826898,0.0,1.0,186.3400
2020-05-26 00:00:00+00:00,181.570,186.50,181.1000,186.3400,36073609,180.236896,185.130700,179.770347,184.971874,36073609,0.0,1.0,180.2000
2020-05-27 00:00:00+00:00,181.810,181.99,176.6000,180.2000,39517146,180.475134,180.653812,175.303386,178.876955,39517146,0.0,1.0,180.7400


In [ ]:
#1=buy, 0=sell
msft_ticker_history['class']=np.where(msft_ticker_history['open_nextday']>msft_ticker_history['open'], 1, 0)
#drop last row b/c open_nextday = NaN
msft_ticker_history.drop(msft_ticker_history.tail(1).index,inplace=True) # drop last row
msft_ticker_history

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,open_nextday,class
date,,,,,,,,,,,,,,
2013-12-16 00:00:00+00:00,36.885,37.00,36.5400,36.7300,31734200,31.826906,31.926135,31.529216,31.693161,31734200,0.0,1.0,36.9400,1
2013-12-17 00:00:00+00:00,36.520,37.11,36.3344,36.9400,45687700,31.511959,32.021051,31.351810,31.874363,45687700,0.0,1.0,36.3601,0
2013-12-18 00:00:00+00:00,36.580,36.60,35.5300,36.3601,63192100,31.563731,31.580988,30.657719,31.373986,63192100,0.0,1.0,36.5100,1
2013-12-19 00:00:00+00:00,36.250,36.55,36.0800,36.5100,34160100,31.278984,31.537845,31.132296,31.503330,34160100,0.0,1.0,36.2000,0
2013-12-20 00:00:00+00:00,36.800,36.93,36.1850,36.2000,62649100,31.753562,31.865735,31.222898,31.235841,62649100,0.0,1.0,36.8100,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-21 00:00:00+00:00,183.430,186.67,183.2900,185.4000,29032741,182.083240,185.299451,181.944268,184.038776,29032741,0.0,1.0,183.1900,0
2020-05-22 00:00:00+00:00,183.510,184.46,182.5400,183.1900,20826898,182.162652,183.105677,181.199774,181.845002,20826898,0.0,1.0,186.3400,1
2020-05-26 00:00:00+00:00,181.570,186.50,181.1000,186.3400,36073609,180.236896,185.130700,179.770347,184.971874,36073609,0.0,1.0,180.2000,0


In [ ]:
for i in range(50):
  msft_ticker_history[str(i) + 'previous day'] = msft_ticker_history['open'].shift(i)



In [ ]:
msft_ticker_history = msft_ticker_history[49:]

In [ ]:
msft_ticker_history

In [ ]:
#data should now be rady for conversion to tensors, and train/test split

#CNN set up

In [ ]:
#library imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

device = torch.device('cpu')
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
# Hyper-parameters 
num_epochs = 5
batch_size = 4
learning_rate = 0.000001

In [ ]:
#Create target tensor
y = msft_ticker_history['class']

#remove class and open_nextday(forward looking variable) columns
msft_no_target = msft_ticker_history.drop(labels=None, axis=0, index=None, columns=["class", "open_nextday"], level=None, inplace=False, errors='raise')

X = msft_no_target
X

In [ ]:
#train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train

In [ ]:
X_train = torch.tensor(X_train.values)
X_test = torch.tensor(X_test.values)
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

In [ ]:
sum(X_train)

In [ ]:
# define transforms
transform = transforms.Compose(
    [
     #transforms.ToTensor(),
     transforms.Normalize((43,), (43,))
     ])

In [ ]:
# custom dataset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class MSFTDataset(Dataset):
    def __init__(self, data, labels, transforms=None):
        
        self.X = data
        self.y = labels
        self.transform = transform
        
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X[i]
        labels = self.y[i]
        
        if self.transform:
            data = self.transform(data.unsqueeze(1).unsqueeze(1))
            data = data.squeeze(1).squeeze(1)

        return data, labels


#datasets
train_data = MSFTDataset(X_train, y_train, transform)
test_data = MSFTDataset(X_test, y_test, transform)

# dataloaders
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

classes = (1, 0)

In [ ]:
dataiter = iter(trainloader)

x = dataiter.next()
x

[tensor([[ 6.8372e-01,  7.1791e-01,  6.8372e-01,  7.1116e-01,  5.0777e+05,
           6.0187e-01,  6.3440e-01,  6.0187e-01,  6.2798e-01,  5.0777e+05,
          -1.0000e+00, -9.7674e-01,  7.1116e-01,  7.0558e-01,  7.1140e-01,
           6.9907e-01,  6.6535e-01,  6.7209e-01,  6.8023e-01,  6.7651e-01,
           6.9302e-01,  6.8372e-01,  6.7884e-01,  6.8721e-01,  7.0000e-01,
           7.0465e-01,  6.9000e-01,  7.1535e-01,  7.2884e-01,  7.1628e-01,
           7.1000e-01,  7.0814e-01,  7.2512e-01,  7.0930e-01,  6.9977e-01,
           6.9302e-01,  6.8000e-01,  6.6279e-01,  6.4395e-01,  6.2791e-01,
           6.1535e-01,  5.9767e-01,  5.8767e-01,  5.8733e-01,  6.1233e-01,
           5.9953e-01,  6.1349e-01,  6.0953e-01,  6.3047e-01,  6.6047e-01,
           6.3000e-01,  6.4047e-01,  6.3279e-01,  6.4698e-01,  6.3953e-01,
           6.2163e-01,  6.1093e-01,  6.4907e-01,  6.2837e-01,  6.1047e-01,
           6.7523e-01,  6.8628e-01],
         [ 1.2428e+00,  1.2493e+00,  1.2214e+00,  1.2377e+00,  

In [ ]:
data, labels = dataiter.next()

In [ ]:
conv1 = nn.Conv1d(1, 6, 5)
pool = nn.MaxPool1d(2, 1)
conv2 = nn.Conv1d(1, 16, 5)

In [ ]:
x = conv1(data.unsqueeze(1).float())
print(x.shape)

torch.Size([1, 6, 58])


In [ ]:
x = pool(x)
print(x.shape)

torch.Size([1, 16, 56])


In [ ]:
x = conv2(data.unsqueeze(1).float())
print(x.shape)

torch.Size([1, 16, 58])


In [ ]:
x = pool(x)
print(x.shape)

torch.Size([1, 16, 57])


In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    #1 = input channels, 6 = output size, 5=kernel size
    self.conv1 = nn.Conv1d(1, 6, 5)
    #2 = kernel size, 1 = stride
    self.pool = nn.MaxPool1d(2, 1)
    #6 = input channel size (same as output of last layer), 16 = output, 5 = kernel size
    self.conv2 = nn.Conv1d(6, 16, 5)
    #16*5*5 = input, 120 = output
    self.fc1 = nn.Linear(16*52, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 2)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    #print(x.shape)
    x = self.pool(F.relu(self.conv2(x)))
    #print(x.shape)
    #-1 finds correct number of samples in batch for us
    x = x.view(-1, 16*52)
    #print(x.shape)
    x = F.relu(self.fc1(x))
    #print(x.shape)
    x = F.relu(self.fc2(x))
    #print(x.shape)
    x = self.fc3(x)
    #print(x.shape)
    return x


In [ ]:
model = ConvNet()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(trainloader)

In [ ]:
for epoch in range(num_epochs):
  for i, (data, labels) in enumerate(trainloader):

    data = data.to(device)
    labels = labels.to(device)

    #forward pass
    print(torch.unsqueeze(data, 1))
    outputs = model((torch.unsqueeze(data, 1)).float())
    loss = criterion(outputs, labels)

    #backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print("Finished Training")

Streaming output truncated to the last 5000 lines.
          -5.1163e-03, -2.7907e-03, -1.0233e-02, -1.7209e-02, -3.6512e-02,
          -3.7907e-02, -3.5581e-02, -5.3488e-02, -3.8837e-02, -1.6047e-02,
          -1.5116e-02, -1.8837e-02,  0.0000e+00,  1.6279e-03,  2.3256e-04,
           1.3023e-02,  1.5581e-02,  2.6279e-02,  2.3023e-02,  2.2093e-02,
           2.6744e-02,  1.6279e-02,  1.1860e-02,  4.1860e-03,  1.4651e-02,
           2.2558e-02,  8.8372e-03, -7.9070e-03, -8.1395e-03, -6.0465e-03,
          -1.7674e-02, -7.4419e-03, -1.8140e-02, -2.4651e-02, -3.1860e-02,
          -5.6047e-02, -3.3721e-02]],

        [[ 2.3595e+00,  2.3726e+00,  2.3467e+00,  2.3714e+00,  4.2442e+05,
           2.3054e+00,  2.3182e+00,  2.2928e+00,  2.3171e+00,  4.2442e+05,
          -1.0000e+00, -9.7674e-01,  2.3714e+00,  2.3681e+00,  2.3549e+00,
           2.3698e+00,  2.3377e+00,  2.3507e+00,  2.3581e+00,  2.2405e+00,
           2.2416e+00,  2.1833e+00,  2.2319e+00,  2.2198e+00,  2.2502e+00,
          

In [ ]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(2)]
  n_class_samples = [0 for i in range(2)]

  for data, labels in testloader:
    data = data.to(device)
    labels = labels.to(device)
    outputs = model((torch.unsqueeze(data, 1)).float())
    print(outputs)

    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(1):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1

  acc = 100.0 * n_correct/n_samples
  print(f'Accuracy of the network: {acc} %')

  for i in range(2):
    acc = 100 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc} %')

tensor([[-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083]])
tensor([[-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083]])
tensor([[-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083]])
tensor([[-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083]])
tensor([[-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083]])
tensor([[-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083]])
tensor([[-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083]])
tensor([[-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083]])
tensor([[-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.0417,  0.0083],
        [-0.